In [20]:
import pickle
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from matplotlib import pyplot
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nana-\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
pd.set_option('display.max_colwidth', -1)
df = pd.read_csv("../data/tweets_1.csv")

In [8]:
x= df['text'][df['Unnamed: 0']==2]
print(x)

2    The media and establishment want me out of the race so badly -  I WILL NEVER DROP OUT OF THE RACE, WILL NEVER LET MY SUPPORTERS DOWN! #MAGA
Name: text, dtype: object


In [9]:
df.head(10)

,Unnamed: 0,date,id,link,retweet,text,author
0,0,Oct 7,784609194234306560,/realDonaldTrump/status/784609194234306560,False,Here is my statement.pic.twitter.com/WAZiGoQqMQ,DonaldTrump
1,1,Oct 10,785608815962099712,/realDonaldTrump/status/785608815962099712,False,Is this really America? Terrible!pic.twitter.com/WiwC61PIFu,DonaldTrump
2,2,Oct 8,784840992734064640,/realDonaldTrump/status/784840992734064641,False,"The media and establishment want me out of the race so badly - I WILL NEVER DROP OUT OF THE RACE, WILL NEVER LET MY SUPPORTERS DOWN! #MAGA",DonaldTrump
3,3,Oct 8,784767399442653184,/realDonaldTrump/status/784767399442653184,False,Certainly has been an interesting 24 hours!,DonaldTrump
4,4,Oct 10,785561269571026944,/realDonaldTrump/status/785561269571026946,False,Debate polls look great - thank you!\r\n#MAGA #AmericaFirstpic.twitter.com/4peQ3Sswdz,DonaldTrump
5,5,4h4 hours ago,786950598826532864,/realDonaldTrump/status/786950598826532864,False,WHAT THEY ARE SAYING ABOUT THE CLINTON CAMPAIGN’S ANTI-CATHOLIC BIGOTRY: \r\nhttp://bit.ly/2dcbtvkCrooked,DonaldTrump
6,6,Oct 13,786589172911964160,/realDonaldTrump/status/786589172911964161,False,Thank you! #MAGA #AmericaFirstpic.twitter.com/fG313wjlKm,DonaldTrump
7,7,Oct 12,786340623804751872,/realDonaldTrump/status/786340623804751872,False,"I will be in Cincinnati, Ohio tomorrow night at 7:30pm- join me! #OhioVotesEarly #VoteTrumpPence16 \r\nTickets: https://www.donaldjtrump.com/schedule/register/cincinnati-oh2/ …pic.twitter.com/XUFuGc4Fg5",DonaldTrump
8,8,Oct 12,786201435486781440,/realDonaldTrump/status/786201435486781440,False,Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So dishonest! Rigged system!,DonaldTrump
9,9,Oct 11,786054986534969344,/realDonaldTrump/status/786054986534969344,False,Thank you Florida- a MOVEMENT that has never been seen before and will never be seen again. Lets get out & #VoteTrumpPence16 on 11/8! #MAGApic.twitter.com/e90a2AL7vO,DonaldTrump


In [10]:
def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

In [11]:
df['text'] = df['text'].astype(str)

In [12]:
df['cleaned'] = df['text'].apply(_removeNonAscii)

df['cleaned'] = df.cleaned.apply(func = make_lower_case)
df['cleaned'] = df.cleaned.apply(func = remove_stop_words)
df['cleaned'] = df.cleaned.apply(func=remove_punctuation)
df['cleaned'] = df.cleaned.apply(func=remove_html)


In [13]:
EMBEDDING_FILE = '../models/GoogleNews-vectors-negative300.bin.gz'
google_word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)


In [14]:
#ssplit en plusieurs mots la description cleaned
import time
corpus = []
for words in df['cleaned']:
    corpus.append(words.split())

In [15]:
# Entrainement surr google model
%time
google_model = Word2Vec(size = 300, window=5, min_count = 2, workers = -1)
google_model.build_vocab(corpus)



google_model.intersect_word2vec_format(EMBEDDING_FILE, lockf=1.0, binary=True)

google_model.train(corpus, total_examples=google_model.corpus_count, epochs = 5)

Wall time: 0 ns


(0, 0)

In [16]:
google_model.wv.most_similar(positive=["steve"]) #model google des mots reconus on peut tester des trucs ex: ok, hello, etc

[('dave', 0.7480320334434509),
 ('jeff', 0.6910731196403503),
 ('jason', 0.6878510117530823),
 ('robert', 0.6830392479896545),
 ('george', 0.6792417764663696),
 ('kevin', 0.676276683807373),
 ('todd', 0.6758198738098145),
 ('ryan', 0.6735630035400391),
 ('greg', 0.670603334903717),
 ('michael', 0.6679339408874512)]

In [17]:
# genere average word2dec pour chaque champ
def vectors(x):
    
    
    global word_embeddings
    word_embeddings = []

    
    for line in df['cleaned']:
        avgword2vec = None
        count = 0
        for word in line.split():
            if word in google_model.wv.vocab:
                count += 1
                if avgword2vec is None:
                    avgword2vec = google_model[word]
                else:
                    avgword2vec = avgword2vec + google_model[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
        
            word_embeddings.append(avgword2vec)


In [31]:


def recommendations(text):
    
  

    vectors(df)
    
 

    cosine_similarities = cosine_similarity(word_embeddings, word_embeddings)

  
    books = df[['text', 'id']]
 
    indices = pd.Series(df.index, index = df['text']).drop_duplicates()

    idx=df[df['text']=='text']
    print(idx)

    #idx = indices[text]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:20]
    book_indices = [i[0] for i in sim_scores]
    recommend = books.iloc[book_indices]

    print(sim_scores)
    return(recommend)

In [ ]:
1854

In [32]:
recommendations("I will be re-tweeting some of your better, most imaginative and hopefully insightful tweets. Make them good (great)! Important stuff.")

1854
[(7135, 0.7707242), (9601, 0.7637521), (9234, 0.7598102), (7543, 0.7593678), (14302, 0.75635105), (16362, 0.75635105), (2441, 0.7557487), (10260, 0.75278187), (9489, 0.7492509), (12573, 0.746034), (5459, 0.7410302), (8931, 0.73876345), (13384, 0.73568606), (15444, 0.73568606), (2474, 0.73501897), (5988, 0.73496044), (6860, 0.7338564), (9272, 0.732393), (4152, 0.73117673)]


,text,id
7135,"""@Natasha_tut13: @realDonaldTrump I like to read your tweets and I am delighted with you, intelligent, decent, strong man!"" Thanks.",480517974944514048
9601,"""@thineline4: @realDonaldTrump For a long time I thought you were so nasty. Now after reading your tweets I changed my mind."" Great!",368563292823752704
9234,"New York Fashion Week is really bad, and used to be so glamorous and exciting! No stars, no fun-just boring. They need serious help. #NYFW",379050677370355712
7543,“No matter how good you get you can always get better and that's the exciting part.” - @TigerWoods,464489915750055936
14302,My twitter account is now reaching more people than the New York Times--not bad. And we're only going to get better!,187573782687391744
16362,My twitter account is now reaching more people than the New York Times--not bad. And we're only going to get better!,187573782687391744
2441,#DemDebate was really boring but had a lot of fun live tweeting and picked up by far the most followers.,654310045304389632
10260,"""@CariRead08: @realDonaldTrump I look forward to your wise words every day. Nice to learn from one of the best."" You are very smart!",349713546445205504
9489,"""@jamieaydt: One day I hope to be successful & a bit controversial like @realDonaldTrump ...keeping things interesting & classy"" Go for it!",372940049815719936
12573,"“I always follow my own instincts, but I am not going to kid you: it’s also nice to get good reviews.” - The Art of the Deal",286527813597679616


In [26]:
with open('../models/google_model.pickle', 'wb') as output:
    pickle.dump(google_model, output)

In [25]:

with open('../models/google_model.pickle', 'rb') as data:
    google_model = pickle.load(data)